## Ansible Playbook - Host, Guest OS and OpenShift Details
This playbook will install the following packages:
- util-linux
- tuned
- perf 
- sysstat 
- tuna 
- numactl

The execute the following commands:
- uname -a
- tee /proc/cmdline
- lscpu
- tuned-adm active

In [ ]:
#inventory
[master]
jcallen-node1.cloud.lab.eng.bos.redhat.com ansible_user=root ansible_ssh_pass=WileyFuse2014
[nodes]
jcallen-node1.cloud.lab.eng.bos.redhat.com ansible_user=root ansible_ssh_pass=WileyFuse2014
[instances]
goldman07 ansible_host=10.130.0.222 ansible_user=cloud-user ansible_ssh_pass=r3dh@t2018
[os:children]
nodes
instances

In [ ]:
#play
name: OpenShift
hosts: master
gather_facts: false

In [ ]:
#task
file:
    dest: "/opt/app-root/openshift.log"
    state: touch
delegate_to: localhost

In [ ]:
#task:
command: oc login -u system:admin

In [ ]:
#task
command: oc describe nodes
register: openshift_output
with_items:
- oc describe nodes
- oc get pods --all-namespaces
- oc get configmaps --all-namespaces
- oc describe vm --all-namespaces
- oc describe vmi --all-namespaces

In [ ]:
#task
lineinfile:
    line: "{{ item.stdout }}"
    dest: "/opt/app-root/openshift.log"
    insertafter: EOF
delegate_to: localhost
with_items: "{{ openshift_output.results }}"

In [ ]:
#play
name: Extract Information
hosts: os
gather_facts: false
vars:
    packages:
    - util-linux
    - tuned
    - perf 
    - sysstat 
    - tuna 
    - numactl 

In [ ]:
#task:
yum:
    name: "{{ packages }}"
    state: latest
become: True

In [ ]:
#task
command: "{{ item }}"
become: True
with_items:
- uname -a
- cat /proc/cmdline
- lscpu
- tuned-adm active
register: output

In [ ]:
#task
file:
    dest: "/opt/app-root/{{ inventory_hostname }}.log"
    state: touch
delegate_to: localhost

In [ ]:
#task
lineinfile:
    line: "{{ item.stdout }}"
    dest: "/opt/app-root/{{ inventory_hostname }}.log"
    insertafter: EOF
delegate_to: localhost
with_items: "{{ output.results }}"